In [298]:
# Dependencies
import pandas as pd
import numpy as np
from postgres_key import mykey
from sqlalchemy import create_engine

In [299]:
# Load csv and rename columns
alchohol = pd.read_csv('./Resources/alcohol.csv')
alchohol_clean = alchohol[['SpatialDimensionValueCode','TimeDim','DisaggregatingDimension1ValueCode','Value']]
alchohol_clean.rename(columns={'SpatialDimensionValueCode':'country_code',
                               'TimeDim':'year',
                               'DisaggregatingDimension1ValueCode':'alc_type',
                               'Value':'alc_num'},inplace=True)
alchohol_clean.head()

c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,country_code,year,alc_type,alc_num
0,DJI,1971,SA_SPIRITS,0.00
1,FIN,1981,SA_WINE,1.07
2,AUT,1987,SA_SPIRITS,1.90
3,GBR,1993,SA_SPIRITS,1.87
4,LUX,1964,SA_WINE,10.34


In [300]:
# Load csv and drop unnecessary columns
income = pd.read_csv('./Resources/income_ppp.csv')

income_clean=income.drop(columns=['Indicator Name','Indicator Code'])

# Transpose the year columns into a single new column of years and drop NaN values
income_clean=income_clean.melt(id_vars=['Country Name','Country Code'],var_name='year',value_name='gdp_per_capita')
income_clean.dropna(inplace=True)

# Rename columns and drop 'country_name'
income_clean.rename(columns={'Country Code':'country_code',
                             'Country Name':'country_name'},inplace=True)
income_clean.drop(columns=['country_name'],inplace=True)
income_clean.loc[income_clean['country_code']=='AUS',:]

,country_code,year,gdp_per_capita
7993,AUS,1990,17367.11058
8259,AUS,1991,17822.47528
8525,AUS,1992,18214.59787
8791,AUS,1993,19159.52159
9057,AUS,1994,20094.05411
9323,AUS,1995,20923.55520
9589,AUS,1996,22008.91159
9855,AUS,1997,22972.48786
10121,AUS,1998,24210.63627
10387,AUS,1999,25296.26355


In [301]:
# Load csv and rename columns
traffic_death = pd.read_csv('./Resources/traffic_death.csv')
traffic_death_clean = traffic_death[['SpatialDimValueCode','Period','FactValueNumeric']]
traffic_death_clean.rename(columns={'SpatialDimValueCode':'country_code',
                               'FactValueNumeric':'deaths_per_100k',
                               'Period':'year'},inplace=True)
traffic_death_clean.head()

c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,country_code,year,deaths_per_100k
0,ATG,2019,0.00
1,FSM,2019,0.16
2,MDV,2019,1.63
3,KIR,2019,1.92
4,EGY,2019,10.10


In [302]:
# Load csv and rename columns
country = pd.read_csv('./Resources/country.csv')
country.rename(columns={'Title':'Country'},inplace=True)
country.drop(columns=['Dimension','ParentDimension','ParentCode','ParentTitle'],inplace=True)
country.head()

,country_code,country_name
0,ABW,Aruba
1,AFE,Africa Eastern and Southern
2,AFW,Africa Western and Central
3,AGO,Angola
4,ALB,Albania


In [303]:
# Rename country columns for merging with median_age
country.rename(columns={'country_code':'Code',
                        'country_name':'Country'
                        },inplace=True)

In [305]:
# Load csv and rename columns and merge with country dataframe
median_age = pd.read_csv('./Resources/median_age.csv')
median_age_clean = median_age.drop(columns=['Variant','Type'])
median_age_clean.rename(columns={'Type of aggregate, group, and constituents *':'Country'},inplace=True)
median_age_clean=median_age_clean.merge(country,how='left',on='Country')

# Find out if any countries have CountryCodes missing
median_age_clean.loc[median_age_clean['Code'].isna(),:]

# Assign missing values to CountryCode
map=["PRK","FSM","TUR","PSE","CHI","HKG","MAC","TWN","GBR","VIR","REU","ESH"]
median_age_clean.loc[median_age_clean['Code'].isna(),"Code"]=map

# Choose columns from 1950 to 2020, and the 'Code' column
median_age_clean=median_age_clean.iloc[:,np.r_[0:16,32]]

# Transpose the year columns into a single new column of years and drop country_name column
median_age_clean=median_age_clean.melt(id_vars=['Country','Code'],var_name='Year',value_name='Median_age')
median_age_clean.rename(columns={'Country':'country_name',
                                 'Code':'country_code',
                                 'Year':'year',
                                 'Median_age':'median_age'},inplace = True)
median_age_clean.drop(columns=['country_name'],inplace=True)
median_age_clean.loc[median_age_clean.country_code=='PRK',:]


,country_code,year,median_age
2,PRK,1950,18.0
203,PRK,1955,19.1
404,PRK,1960,19.9
605,PRK,1965,21.1
806,PRK,1970,21.0
1007,PRK,1975,19.6
1208,PRK,1980,21.8
1409,PRK,1985,23.5
1610,PRK,1990,25.0
1811,PRK,1995,27.3


In [306]:
# Rename country columns to match tables in database
country.rename(columns={'Code':'country_code',
                        'Country':'country_name'
                        },inplace=True)

In [311]:
# Load dataframe into SQL database
database_key = f""
engine = create_engine(f'postgresql://{mykey}@localhost:5432/project_4')
country.to_sql(name='country', con=engine, if_exists='append', index=False)

In [312]:
# Load dataframe into SQL database
database_key = f""
engine = create_engine(f'postgresql://{mykey}@localhost:5432/project_4')
median_age_clean.to_sql(name='median_age', con=engine, if_exists='append', index=False)

In [313]:
# Load dataframe into SQL database
database_key = f""
engine = create_engine(f'postgresql://{mykey}@localhost:5432/project_4')
alchohol_clean.to_sql(name='alcohol', con=engine, if_exists='append', index=False)

In [314]:
# Load dataframe into SQL database
database_key = f""
engine = create_engine(f'postgresql://{mykey}@localhost:5432/project_4')
income_clean.to_sql(name='income', con=engine, if_exists='append', index=False)

In [315]:
# Load dataframe into SQL database
database_key = f""
engine = create_engine(f'postgresql://{mykey}@localhost:5432/project_4')
traffic_death_clean.to_sql(name='traffic_deaths', con=engine, if_exists='append', index=False)

In [316]:
# Checking tables in database
engine.table_names()

c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['country', 'median_age', 'alcohol', 'income', 'traffic_deaths']

In [317]:
# Checking country table to ensure data loaded correctly
pd.read_sql_query('select * from country', con=engine).head()

,country_code,country_name
0,ABW,Aruba
1,AFE,Africa Eastern and Southern
2,AFW,Africa Western and Central
3,AGO,Angola
4,ALB,Albania
